credits to the YOLO v4 repository that I use as the backbone of this.

In [ ]:
import numpy as np
import pandas as pd
import rosbag
import os
import yaml
import random
import math as m
from scipy.spatial.distance import cdist
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error

ganti yaml tiap awal generasi

In [ ]:
#function for changing the parameter values (YAML file is used for the params to be called from the main UKF program)
def tes_yaml(k,P,Q):
    with open('/home/win/awin_ws/src/robot_localization/awin/params/ukf_ule_ga_params.yaml') as tes:             
        doc=yaml.safe_load(tes)
    if Q is not None:
        doc['kappa'] = k
        doc['process_noise_covariance_Q'] = Q
        doc['initial_estimate_covariance_P'] = P
   
    with open('/home/win/awin_ws/src/robot_localization/awin/params/ukf_ule_ga_params.yaml','w') as tes:      #save as
        yaml.safe_dump(doc, tes, indent=2, allow_unicode=False) #dah berhasil ni     

jalanin roslaunch berisi rosbag play, ukf dengan yaml baru untuk setiap generasi, dan rosbag record

In [ ]:
#function for launching the 3 nodes: rosbag play back, the ukf algorithm, the recording of the rosbag to a temporary rosbag file that saves the estimation from the UKF
def launch():
    !roslaunch /home/win/awin_ws/src/robot_localization/awin/ukf_ule_ga.launch

jalanin rosbag import cari rmse

In [ ]:
#function for loading the estimation values recorded in the temporary bag file
def ambil_bag():
    bag = rosbag.Bag('ga_temp.bag')
    states=[]
    states_baru=[]
    states_baru_plus1=[]

    for topic, msg, t in bag.read_messages(topics=['/ukf_states']):
        states.append([msg.x,msg.y])
    states = np.array(states)
    
    states=states[np.all(states!=0,axis=1)]         #cleaning out value of 0
    states=states[np.all(~np.isnan(states),axis=1)] #cleaning out value of NaN

    utm=[]
    utm_baru=[]
    for topic, msg, t in bag.read_messages(topics=['/utm']):
        utm.append([msg.pose.pose.position.x,msg.pose.pose.position.y])
    utm = np.array(utm)

    min=np.amin(states,axis=0)

    i=0
    for a in states:
        states_baru.append([states[i,0]-min[0],states[i,1]-min[1]])
        i=i+1
    states_baru=np.array(states_baru)
    
    i=1
    for a in states:
        states_baru_plus1.append([states[i,0]-min[0],states[i,1]-min[1]])
        i=i+1
        if i==len(states):
            i=0
    states_baru_plus1=np.array(states_baru_plus1)
    
    i=0
    for a in utm:
        utm_baru.append([utm[i,0]-min[0],utm[i,1]-min[1]])
        i=i+1
    utm_baru=np.array(utm_baru)
    return states_baru,utm_baru,states_baru_plus1

#function for returning the error (RMSE and MAE) values of the estimation
def er(estimasi,ground_truth,utm):
    def closest_node(node, nodes):
        return nodes[cdist([node], nodes).argmin()]
    
    closest=[]
    i=0
    for row in estimasi:
        closest.append(closest_node(estimasi[i],ground_truth)) #'lingkaran' sebagai target ground truth, bisa diganti
        i=i+1
        
    closest_to_utm=[]
    i=0
    for row in estimasi:
        closest_to_utm.append(closest_node(estimasi[i],utm)) #'lingkaran' sebagai target ground truth, bisa diganti
        i=i+1

    #RMSE
    rmse=sqrt(mse(closest,estimasi))
    rmse_to_utm=sqrt(mse(closest_to_utm,estimasi))
    print('rmse = '+str(rmse)+" meter"+"\nrmse to utm = "+str(rmse_to_utm)+" meter")
    #MAE
    mae=mean_absolute_error(closest,estimasi)
    return rmse,mae,rmse_to_utm

#function for extending the points of the GNSS (UTM) points by means of linearization so that it is almost continuous-like
def ndl(x1, y1, x2, y2,selisih):
    dy=y2-y1
    dx=x2-x1
    m = float(y2 - y1)/float(x2 - x1)
    if(x1>x2):
        x1,x2,y1,y2 = x2,x1,y2,y1

    x=x1
    y=y1
    if abs(dx)>abs(dy):
        while x<x2-selisih:
            x=selisih+x
            y = m*selisih + y    
            list.append([x,y])
    if abs(dy)>abs(dx):
        if(y1>y2):
            x1,x2,y1,y2 = x2,x1,y2,y1
            x=x1
            y=y1
        while y<y2-selisih:
            x=selisih/m+x
            y = selisih + y    
            list.append([x,y])

#function for creating the list of the extended points of the GNSS
def hubungin_garis(dihubung,selisih):
    i=0
    while range(len(dihubung)):
        if i==len(dihubung)-1:
            break
        ndl(dihubung[i,0],dihubung[i,1],dihubung[i+1,0],dihubung[i+1,1],selisih)
        i=i+1
    return list

In [ ]:
#function for fitness evaluation
def fitness(rmse_groundtruth,rmse_titikutm):
    t=1
    if rmse_titikutm<0.0605: #value is acquired from observation
        t=0
    return t/rmse_groundtruth

In [ ]:
#function for plotting the estimation and the (unextended) GNSS points 
def plot(estimasi,ground_truth):
    plt.figure(figsize=(10,10))
    matplotlib.style.use('default')

    plt.scatter(estimasi[:,0],estimasi[:,1],s=2.,color='C7') #odometry/filtered_map
    plt.scatter(ground_truth[:,0],ground_truth[:,1],s=5.,c='C3',alpha=1)
    plt.show()

GA

In [ ]:
#function for saving the acquired parameter values to an external .txt file
def catat(ktemp,ptemp,qtemp,errortemp,fitnesstemp,k_path,p_path,q_path,error_path,error_bygen_path,fitness_path):
    p='%14.4g' * len(ptemp) % tuple(ptemp)
    q='%14.4g' * len(qtemp) % tuple(qtemp)
    k=str(ktemp)
    error=str(errortemp)
    fitness=str(fitnesstemp)
    
    with open(k_path, 'a') as h:  
        h.write(k + '\n')
    with open(p_path, 'a') as h:  
        h.write(p + '\n')
    with open(q_path, 'a') as h: 
        h.write(q + '\n')
    with open(error_path, 'a') as h:  
        h.write(error + '\n')
    with open(error_bygen_path, 'a') as h:  
        h.write(error + '\n')
    with open(fitness_path, 'a') as h:  
        h.write(fitness + '\n')
        
    x_k=np.loadtxt(k_path, ndmin=2)  
    x_q=np.loadtxt(q_path, ndmin=2)  
    x_p=np.loadtxt(p_path, ndmin=2) 

    x_error=np.loadtxt(error_path, ndmin=2)
    x_error_bygen=np.loadtxt(error_bygen_path, ndmin=2)
    x_fitness=np.loadtxt(fitness_path, ndmin=2)
    x_fitness_f=x_fitness.flatten()
    
    np.savetxt(k_path, x_k[np.argsort(-x_fitness_f)], '%10.5g')
    np.savetxt(p_path, x_p[np.argsort(-x_fitness_f)], '%10.5g')
    np.savetxt(q_path, x_q[np.argsort(-x_fitness_f)], '%10.5g')
    np.savetxt(error_path, x_error[np.argsort(-x_fitness_f)], '%10.5g')
    np.savetxt(error_bygen_path, x_error_bygen, '%10.5g') #savenya berdasarkan error, loadnya berdasarkan fitness
    np.savetxt(fitness_path, x_fitness[np.argsort(-x_fitness_f)], '%10.5g')  #[np.argsortnya ini nambah dimensi]

In [ ]:
#initializing the parameter values
k_temp=-2
p_temp=[1e-2, 0, 0, 0, 0,
        0, 1e-2, 0, 0, 0,
        0, 0, 0.1, 0, 0,
        0, 0, 0, 0.1, 0,
        0, 0, 0, 0, 1e-3]
q_temp=[10, 0, 0, 0, 0,
         0, 10, 0, 0, 0,
         0, 0, 1, 0, 0,
         0, 0, 0, 1, 0,
         0, 0, 0, 0, 0.1]

generasi=1000 #define the number of generations

#defining the path for the parameter values to be saved to
k_path = "/home/win/ga/path_GA/coba_3/k_path.txt"
p_path = "/home/win/ga/path_GA/coba_3/p_path.txt"
q_path = "/home/win/ga/path_GA/coba_3/q_path.txt"
fitness_path = "/home/win/ga/path_GA/coba_3/fitness_path.txt"
error_path = "/home/win/ga/path_GA/coba_3/error_path_byfitness.txt"
error_bygen_path = "/home/win/ga/path_GA/coba_3/error_bygen.txt"

daftar_error=[]
daftar_w=[]
daftar_fe=[]
daftar_parameter=[]

p_ortu=[]
q_ortu=[]
for i in range(generasi):
    print("----------------------------------------------------------------------------------------------------")
    print("Generasi ke-"+ str(i+1) + " dari " + str(generasi))

    if os.path.exists(p_path):
        k = np.loadtxt(k_path, ndmin=2)
        p = np.loadtxt(p_path, ndmin=2)
        q = np.loadtxt(q_path, ndmin=2)
        error = np.loadtxt(error_path, ndmin=2)
        fitness = np.loadtxt(fitness_path, ndmin=2)

        #selection
        n = min(10, len(p)) #defining the number of the individuals in the mating pool
        fitness_f=fitness.flatten()
        
        k=k[np.argsort(-fitness_f)][:n] #load dari txt
        p=p[np.argsort(-fitness_f)][:n] #load dari txt
        q=q[np.argsort(-fitness_f)][:n]
        error=error[np.argsort(-fitness_f)][:n]
        fitness=fitness[np.argsort(-fitness_f)][:n]

        #crossover (unweighted average)
#         w = fitness - fitness.min()
        if i>1: #picking the parents for the crossover
            indeks_ortu1,indeks_ortu2=random.sample(range(0, n), 2)
            p_ortu=p[[indeks_ortu1,indeks_ortu2],:]
            q_ortu=q[[indeks_ortu1,indeks_ortu2],:]
            k_ortu=k[[indeks_ortu1,indeks_ortu2]]
            error_ortu=error[[indeks_ortu1,indeks_ortu2],:]
            fitness_ortu=fitness[[indeks_ortu1,indeks_ortu2],:]
            
            k=(k_ortu[0]+k_ortu[1])/2
            p=(p_ortu[0]+p_ortu[1])/2
            q=(q_ortu[0]+q_ortu[1])/2

        #mutation
        mp,s=0.2,3 #mutation probability and the multiplying factor defined
        if i<5: #the initial population number defined here
            mp,s=1,1 #for initial population, mutation is used to randomly pick the values around the inital values defines before
        npr = np.random
        pengali=[]
        for a in range(51):
            pengali.append(abs((npr.random()<mp)*(-0.5+npr.random())*2*s+1)) #diganti biar gausa pake distribusi gauss

        m_p=np.ones(25)
        m_q=np.ones(25)

        for b in range(25):
            m_p[b]=m_p[b]*pengali[b]

        for c in range(25):
            m_q[c]=m_q[c]*pengali[c+25]
            
        k_temp=k*pengali[50]
        k_temp=float(k_temp)
        p_temp = (p*m_p).flatten().tolist()
        q_temp = (q* m_q).flatten().tolist()
    
    #change the value of the parameter with the ones just acquired
    tes_yaml(p_temp,q_temp)
    #run the launch function
    launch()
    
    #acquiring the fitness value by taking the estimation value from the temporary bag file (ga_temp.bag) and running the fitness evaluation function
    hasil_ukf=ambil_bag()[0]
    utm=ambil_bag()[1]
    list=[]
    utm_terhubung=np.array(hubungin_garis(utm,0.01)) #harus ada list=[] di atasnya
    rmse,mae,rmse_to_utm=er(hasil_ukf,utm_terhubung,utm)[0:3] 
#     def fitness(rmse_groundtruth,rmse_titikutm):
#         t=1
#         if rmse_titikutm<0.0605:
#             t=0
#         return t/rmse_groundtruth
    fit=fitness(rmse,rmse_to_utm)
    
    #plot the estimation for every set of parameter values acquired
    plot(hasil_ukf,utm)
    
    #saving the parameter values acquired
    catat(k_temp,p_temp,q_temp,rmse,fit,k_path,p_path,q_path,error_path,error_bygen_path,fitness_path)